<a href="https://colab.research.google.com/github/theforces/Hand-Gesture-Recognition/blob/main/Proiect_TACAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install opencv-python ipyplot

In [ ]:
import cv2 #used to load the images (in the future it will have other uses)
import numpy as np
import glob #used to read image names
import random #used to generate tran/test split
import ipyplot #it will be explained
from matplotlib import pyplot as plt #for plotting needs
import pickle


        WARNING! Google Colab Environment detected!
        You might encounter issues while running in Google Colab environment.
        If images are not displaying properly please try setting `base_64` param to `True`.
        


In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
# Read with glob the images from DB
base_path = "/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/"
files_list = []
for i in range(4):
  files_list.append(glob.glob(base_path+str(i)+"/*.jpg"))
  print(glob.glob(base_path+str(i)+"/*.jpg"))

['/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/0/0_1.jpg', '/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/0/0_2.jpg', '/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/0/0_3.jpg', '/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/0/0_4.jpg', '/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/0/0_5.jpg', '/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/0/0_6.jpg', '/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/0/0_7.jpg', '/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/0/0_8.jpg', '/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/0/0_9.jpg', '/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/0/0_10.jpg']
['/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/1/1_1.jpg', '/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/1/1_2.jpg', '/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/1/1_3.jpg', '/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/1/1_4.jpg', '/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/1/1_5.jpg', '/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/1/1_6.jpg', '/gdr

In [ ]:
def segmentation():
  #Segmentare
  kernel = np.ones((10, 10), np.float32) / 100
  gl = glob.glob('/gdrive/MyDrive/Colab Notebooks/Files/Own_Images/*/*.jpg')
  X = []
  Y = []
  for img_path in gl:
    read_image = cv2.imread(img_path)
    img = read_image[:, :, 2] #se ia un singur canal ca sa se faca distinctia clara intre mana si perete(in cazul aceste se ia rosu deoarece produece cele mei bune rezultateș alb,verde produc 6 rezultate bune,spre deosebire de 9 la rosu)
    img2 = cv2.filter2D(img, -1, kernel)# ca sa se faca imbinarea clara intre ce este mana si ce este fundal(adica sa se stie concret care este mana, care este fundalul), este un filtru de netezire(face blur la imagine folosind media aritmetica)
    ret, th1 = cv2.threshold(img2, 195, 255, cv2.THRESH_BINARY_INV)#pragul de 195 s-a gasit empiric; 255 = valoarea maxima care va fi data fiecarui pixel care depaseste pragul; cv2.THRESH_BINARY = dst(x,y)={maxval if src(x,y)>thresh, 0 otherwise}
                                                                   #trebuie luati cei mai intunecati pixeli
    #th1 = cv2.erode(th1, kernel, iterations=1)
    th1 = cv2.dilate(th1, kernel, iterations=1)
    th1 = cv2.morphologyEx(th1, cv2.MORPH_CLOSE, kernel)#dilatare intai, apoi erodare

    #pana aici s-a obtinut masca, urmeaza partea de extragere a mainii

    # plt.figure()
    # plt.axis("off")
    # plt.imshow(th1, cmap='gray')
    # plt.show()
    # exit()

    #Crop
    contours, hier = cv2.findContours(th1, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)# returneaza conturul mainii pe th1
    size = 0
    hand = []
    for c in contours:
        x, y, w, h = cv2.boundingRect(c) #x,y =  dreptunghiul minim care incadreaza conturul(mana)
        if size < w * h: # se ia cel mai mare obiect(mana)
            hull = cv2.convexHull(c)#obtinerea coordonatelor punctelor pe baza trasarii poligonului
            size = w * h
            hand = img[y:y + h, x:x + w] #se face crop la mana
            hand_bw = th1[y:y + h, x:x + w]#se face crop la imaginea alb negru
            hand[hand_bw == 0] = 0 # s-a luat din imaginea BGR, si ne intereseaza pixeli non zero(fundalul se face negru, mana se face alba)
            #resized_hand = cv2.resize(hand, (128,512), interpolation = cv2.INTER_AREA)
    resized_hand = cv2.resize(hand, (128, 256), interpolation=cv2.INTER_AREA)

    # plt.figure()
    # plt.axis("off")
    # plt.imshow(resized_hand, cmap='gray')
    # plt.show()
    # exit()

    #Hog
    winSize = (64, 64)
    blockSize = (16, 16)
    blockStride = (8, 8) #cat de mult se misca blocul stg-dr(in cazul asta se muta cu jumate)
    cellSize = (8, 8)# is the size of the cells of the img patch over which to calculate the histograms
    nbins = 9 #folosit la histograma, sa vada cate intervale sa puna
    derivAperture = 1
    winSigma = 4.
    histogramNormType = 0
    L2HysThreshold = 2.0000000000000001e-01
    gammaCorrection = 0 #se foloseste at cand se cunoaste tipul bazei de date si sa se faca imaginile cat mai similare una cu alta
    nlevels = 64 #cate detectii sa faca
    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins, derivAperture, winSigma,
                                histogramNormType, L2HysThreshold, gammaCorrection, nlevels) #hog=histograma gradientilor orientati=descriptor care imparte in mai multe zone si face 
                                                                                             #histograma pe fiecare zona a gradientilor; gradient = directia de schimbare a valorilor pixelilor
                                                                                             #gradient pozitiv daca merg de la inchis la deschis, negativ invers
                                                                                             #orientarea gradientilor= teta = arctag(gy/gx) care e in intervalul (0,180 ) grade
                                                                                             #valoarea gradientului = sqrt(gx^2 + gy^2)
                                                                                             #gradientul se calculeaza pe verticala si orizontala(Sobel) pt fiecare pixel=se verifica intensitatea pe verticala si orizontala cu Sobel
    winStride = (8, 8)
    padding = (8, 8)
    locations = ((10, 20),)
    hist = hog.compute(resized_hand, winStride, padding, locations)
    X.append(hist)
    val = "/"
    Y.append(int(img_path[img_path.rfind(val) + 1]))#se foloseste rfind() pt a gasi ultima aparitie a lui / pt a se gasi eticheta(care e primul caracter dupa ultimul /)
    
  return (X, Y)
      
X, Y = segmentation() #  X = descriptors, Y = labels
  

In [ ]:
from sklearn import svm 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
clf = svm.SVC(decision_function_shape='ovo', kernel = "poly", degree = 3) #svm = clasificator binar, avem un polinom de gradul 3 in cazul de fata
                                                                      #comparativ cu KNN unde tii toate datele si vezi fata de care esti mai aproape, la SVM se cauta marginea=linia=vectorul bidim. care e cel mai 
                                                                      #departat de cele 2 clase (practic se cauta unde se poate trage o linie intre 2 clase astfel incat sa fie maxim departata de cele 2 clase), 
                                                                      #iar cand se vrea sa se verifice dupa, doar ne uitam de ce parte a liniei este ceea ce ne intereseaza                    
                                                                      #ca sa poata fi folosit la mai multe clase avem 
                                                                      #OVO(unde se incearca clasa cu clasa, si se cauta unde e distanta cea mai mare si cea cu distanta cea mai mare castiga); 
                                                                      
                                                                      #OVR(pt fiecare clasa se ia clasa de interes si din toate celelalte se face o alta clasa si se cauta vectorii suport)

print(np.shape(X_train))
X_train = np.array(X_train)
print(np.shape(X_train))
X_train = np.squeeze(X_train)
print(np.shape(X_train))

clf.fit(X_train, y_train) #se face antrenarea pe X_train

rez = 0
for i in range(len(X_test)): #se face predictia pe X_test
  
  variable = clf.predict(X_test[i].reshape(1,1764))

  print(y_test[i], variable)#ce trebuie sa fie, ce trebuie sa raspunda

  if y_test[i] == variable:
    rez += 1

print(rez)
print(y_train, y_test)


(26, 1764, 1)
(26, 1764, 1)
(26, 1764)
1 [3]
1 [2]
1 [3]
2 [2]
0 [0]
1 [1]
3 [3]
2 [2]
3 [3]
0 [0]
2 [3]
0 [0]
1 [0]
3 [3]
9
[3, 0, 1, 2, 0, 3, 0, 1, 0, 2, 2, 0, 3, 3, 0, 3, 2, 3, 1, 2, 1, 2, 0, 1, 2, 3] [1, 1, 1, 2, 0, 1, 3, 2, 3, 0, 2, 0, 1, 3]
